In [78]:
from dataset import read_dataset, split
from pathlib import Path
import numpy as np
import logging

In [79]:
data_dir = Path("../data/data.csv")

logging.basicConfig(
    level=logging.INFO,  # Set the logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
df = read_dataset(path=data_dir)
train_df, valid_df, test_df = split(df, data_dir, "InvoiceNo", data_dir)

In [83]:
train_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
1,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
2,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,5,2010-12-01 08:34:00,1.69,13047,United Kingdom
3,536367,22745,POPPY'S PLAYHOUSE BEDROOM,5,2010-12-01 08:34:00,2.10,13047,United Kingdom
4,536367,22748,POPPY'S PLAYHOUSE KITCHEN,5,2010-12-01 08:34:00,2.10,13047,United Kingdom
...,...,...,...,...,...,...,...,...
236018,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
236019,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
236020,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
236021,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


In [82]:
basket

InvoiceNo,536366,536367,536369,536371,536372,536374,536375,536376,536377,536378,...,581538,581566,581567,581570,581578,581580,581582,581583,581585,581587
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10080,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10120,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10123C,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10124A,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
DOT,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
M,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [65]:
from sklearn.neighbors import NearestNeighbors

# Define the number of neighbors to consider (k)
k_neighbors = 3

# Initialize the k-NN model
knn = NearestNeighbors(n_neighbors=k_neighbors, metric="cosine")

# Fit the k-NN model on the similarity matrix
knn.fit(basket)

# Neighboors
neighbor_scores, neighbor_indices = knn.kneighbors(basket, n_neighbors=k_neighbors + 1)
neighbor_indices = pd.DataFrame(neighbor_indices, index=basket.index)


for i in range(0, 4):
    neighbor_indices.iloc[:, i] = neighbor_indices.iloc[:, i].map(
        dict(zip(list(range(len(basket))), basket.index))
    )

neighbor_scores = pd.DataFrame(neighbor_scores, index=basket.index)

In [72]:
neighbor_scores

,0,1,2,3
StockCode,,,,
10002,0.000000e+00,0.727834,0.807550,0.807550
10080,0.000000e+00,0.786799,0.801727,0.802614
10120,0.000000e+00,0.695918,0.711325,0.711325
10123C,0.000000e+00,0.741801,0.781782,0.792610
10124A,0.000000e+00,0.422650,0.591752,0.591752
...,...,...,...,...
C2,1.110223e-16,0.776393,0.776393,0.776393
DOT,0.000000e+00,0.000000,0.292893,0.292893
M,0.000000e+00,0.828037,0.831319,0.836243


In [73]:
bundles = set()

max_distance_threshold = 0.5
for i in range(len(basket)):
    bundle = set()
    for score, idx in zip(neighbor_scores.iloc[i, 1:], neighbor_indices.iloc[i, 1:]):
        if score < max_distance_threshold:
            bundle.add(idx)
    bundles.add(frozenset(bundle))
    


In [76]:
bundles = frozenset({bundle for bundle in bundles if len(bundle) > 1})

In [23]:
text_df.iloc[82]["Description"], text_df.iloc[76]["Description"], text_df.iloc[79]["Description"]

('4 pink dinner candle silver flock',
 '4 blue dinner candles silver flock',
 '4 lavender botanical dinner candles')

In [20]:
neighbor_indices

array([[   0,   82,   76,   79],
       [   1, 1389,   95, 2322],
       [   2,  995, 3327, 1796],
       ...,
       [3894, 3504, 1623, 3358],
       [3895, 1726,  657, 3273],
       [3896, 3889, 1727, 1722]])

In [ ]:
product_id = "22423"

In [ ]:
def get_bundle(product_id, basket, neighbor_indices):
    row_index = basket.index.get_loc(product_id)
    return np.array([basket.index[idx] for idx in neighbor_indices[row_index]])

In [ ]:
bundle = get_bundle(product_id, basket, neighbor_indices)

In [ ]:
def get_description(product_id, df):
    return df[df["StockCode"] == product_id]["Description"].iloc[0]

for product_id in bundle:
    print(get_description(product_id, train_df))

regency cakestand 3 tier
lipstick pen red
toothpaste tube pen
lipstick pen fuschia
